# NRRD to NIfTI Conversion Tutorial (with Orientation Preservation)

This notebook demonstrates how to convert `.nrrd` volumes and segmentation files into  
`.nii.gz` format while **fully preserving spatial orientation** using *space directions* and *space origin*.

It also includes:

### ✔ Separation of segmentation labels  
### ✔ Preservation of spatial metadata  
### ✔ Clean dataset organization  
### ✔ Automatic processing of all subjects

This notebook is intended for public GitHub release and scientific reproducibility.



In [1]:
import os
import nrrd
import nibabel as nib
import numpy as np


## 🔧 NRRD → NIfTI Conversion Function

NRRD files store spatial orientation using:

- **`space directions`** → a 3×3 matrix defining voxel axes in real space  
- **`space origin`** → the world coordinate of the first voxel  

To ensure correct conversion, we reconstruct the NIfTI affine matrix using these fields.

The following function converts raw NRRD arrays into properly oriented NIfTI images.


In [2]:
def nrrd_to_nifti(nrrd_data, nrrd_header):
    """
    Converts an NRRD volume into a NIfTI image while preserving the original
    spatial orientation using 'space directions' and 'space origin'.

    Parameters
    ----------
    nrrd_data : np.ndarray
        The volume array from the NRRD file.
    nrrd_header : dict
        Header containing spatial metadata.

    Returns
    -------
    nib.Nifti1Image
        Volume stored in NIfTI format with correct affine orientation.
    """

    # Orientation vectors
    space_directions = np.array(nrrd_header["space directions"].tolist())
    space_origin = np.array(nrrd_header["space origin"])

    # Build affine matrix
    affine = np.eye(4)
    affine[:3, :3] = space_directions
    affine[:3, 3] = space_origin

    return nib.Nifti1Image(nrrd_data, affine)


## 📁 Input and Output Folders

Set the path to your dataset.  
This script expects the following structure inside each subject folder:

<subject>/3DUS/GAS/GAS.nrrd

<subject>/3DUS/GAS/Segmentation.seg.nrrd

Output will be automatically stored in:

./images/ → converted ultrasound volumes

./labels/ → cleaned segmentation masks (one file per label)

If needed, adapt the folder structure in the code.


In [3]:
# Root dataset directory
root_dir = "THPCA 3DUS DATA - LOUISE"

# Create output folders
os.makedirs("./images", exist_ok=True)
os.makedirs("./labels", exist_ok=True)


## 🔄 Processing All Subjects

The following cell:

1. Iterates through every subject folder  
2. Loads the ultrasound `.nrrd` file (`GAS.nrrd`)  
3. Loads the segmentation file (`Segmentation.seg.nrrd`)  
4. Converts both to NIfTI  
5. Splits segmentation into two separate masks:
   - **Label 1 → GM**
   - **Label 2 → GL**
6. Saves each output file with clear naming conventions

Missing files are reported but do not stop the script.


In [4]:
for name in os.listdir(root_dir):

    folder = os.path.join(root_dir, name, "3DUS", "GAS")
    if not os.path.isdir(folder):
        continue

    gas_path = os.path.join(folder, "GAS.nrrd")
    seg_path = os.path.join(folder, "Segmentation.seg.nrrd")

    print(f"\n--- Processing {name} ---")

    # ----------------------------------------------------------
    # 1. Convert US image
    # ----------------------------------------------------------
    if os.path.exists(gas_path):
        gas_data, gas_header = nrrd.read(gas_path)
        nii_img = nrrd_to_nifti(gas_data, gas_header)
        nib.save(nii_img, f"./images/{name}_GM_3DUS.nii.gz")
        print("   ✓ Converted GAS.nrrd → NIfTI")
    else:
        print("   ⚠️ Missing GAS.nrrd")

    # ----------------------------------------------------------
    # 2. Convert segmentation + split labels
    # ----------------------------------------------------------
    if os.path.exists(seg_path):
        seg_data, seg_header = nrrd.read(seg_path)

        # Label masks (uint8)
        label_GM = (seg_data == 1).astype(np.uint8)
        label_GL = (seg_data == 2).astype(np.uint8)

        # Orientation-correct NIfTI
        nii_GM = nrrd_to_nifti(label_GM, seg_header)
        nii_GL = nrrd_to_nifti(label_GL, seg_header)

        # Save
        nib.save(nii_GM, f"./labels/{name}_GM_seg.nii.gz")
        nib.save(nii_GL, f"./labels/{name}_GL_seg.nii.gz")

        print("   ✓ Segmentation converted and labels separated")
    else:
        print("   ⚠️ Missing Segmentation.seg.nrrd")


FileNotFoundError: [Errno 2] No such file or directory: 'THPCA 3DUS DATA - LOUISE'

## ✅ Conversion Complete

All available NRRD files have been successfully processed.

Your converted dataset is now stored in:

- `./images/` for ultrasound volumes  
- `./labels/` for segmentation masks  

Each segmentation has been separated into individual GM and GL labelmaps in NIfTI format.

This notebook ensures:
- proper spatial orientation handling
- reproducible preprocessing steps
- consistent naming conventions

You can now safely use these files for downstream analysis or neural network training.
